## This is the second part of the Telcom Churn Analysis

In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [23]:
from sklearn.preprocessing import StandardScaler,RobustScaler,QuantileTransformer
from sklearn.feature_selection import SelectKBest,mutual_info_classif
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

In [24]:
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

In [25]:
df = pd.read_csv("/content/drive/MyDrive/DSC 550/data.csv")
df.head()

,SeniorCitizen,tenure,MonthlyCharges,TotalCharges,Churn,gender_Male,Partner_Yes,Dependents_Yes,PhoneService,NonMult_PhoneService,...,NoStreamTV,StreamTV,NoStreamMovie,StreamingMovies,OneYrContract,TwoYrContract,PaperLessBilling,CreditCardPayment,ElectCheckPayment,MailedChkPayment
0,0,1,29.85,29.85,0,0,1,0,0,1,...,0,0,0,0,0,0,1,0,1,0
1,0,34,56.95,1889.50,0,1,0,0,1,0,...,0,0,0,0,1,0,0,0,0,1
2,0,2,53.85,108.15,1,1,0,0,1,0,...,0,0,0,0,0,0,1,0,0,1
3,0,45,42.30,1840.75,0,1,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0
4,0,2,70.70,151.65,1,0,0,0,1,0,...,0,0,0,0,0,0,1,0,1,0


In [26]:
df['Churn'].value_counts()

0    5163
1    1869
Name: Churn, dtype: int64

In [27]:
X = df.drop('Churn',axis=1)
y = df['Churn']

In [28]:
undersample = RandomUnderSampler(sampling_strategy='majority')
X_over,y_over = undersample.fit_resample(X,y)

In [29]:
y_over.value_counts()

0    1869
1    1869
Name: Churn, dtype: int64

In [30]:
from sklearn.model_selection import train_test_split

In [31]:
# Get X_train, X_test, y_train and y_test

X_train, X_test, y_train, y_test = train_test_split(X_over, y_over, test_size=0.20, random_state=42)

Setting up a pipeline

In [32]:
pipe = Pipeline([('scaler',StandardScaler()),
                 ('selector',SelectKBest(mutual_info_classif,k=7)),
                 ('classifier',LogisticRegression())])

Define the search Space

In [33]:
search_space = [{'classifier':[LogisticRegression()],
                 'classifier__penalty':['l1','l2'],
                 'classifier__C':np.logspace(0,4,10)},
                {'classifier':[KNeighborsClassifier()],
                'classifier__n_neighbors':[3,5,8,11],
                 'classifier__weights':['uniform','distance']},
                {'classifier':[RandomForestClassifier()],
                 'classifier__n_estimators':[10,100,1000],
                 'classifier__max_features':[1,2,3]}]

In [34]:
# Run the gridsearch

model = GridSearchCV(pipe,search_space,cv=10,verbose=0)
model.fit(X_train,y_train)


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
100 fits failed out of a total of 370.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
100 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/pipeline.py", line 394, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py", line 1461, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('selector',
                                        SelectKBest(k=7,
                                                    score_func=<function mutual_info_classif at 0x7f14623fd0e0>)),
                                       ('classifier', LogisticRegression())]),
             param_grid=[{'classifier': [LogisticRegression(C=3593.813663804626)],
                          'classifier__C': array([1.00000000e+00, 2.78255940e+00, 7.74263683e+00, 2.15443469e+01,
       5...6810054e+02, 4.64158883e+02, 1.29154967e+03,
       3.59381366e+03, 1.00000000e+04]),
                          'classifier__penalty': ['l1', 'l2']},
                         {'classifier': [KNeighborsClassifier()],
                          'classifier__n_neighbors': [3, 5, 8, 11],
                          'classifier__weights': ['uniform', 'distance']},
                         {'classifier': [

Obtain predictions for train and test sets

In [35]:
best_estm = model.best_estimator_
print(best_estm)

Pipeline(steps=[('scaler', StandardScaler()),
                ('selector',
                 SelectKBest(k=7,
                             score_func=<function mutual_info_classif at 0x7f14623fd0e0>)),
                ('classifier', LogisticRegression(C=3593.813663804626))])


In [36]:
from sklearn.metrics import mean_squared_error

In [37]:
y_pred_train = best_estm.predict(X_train)
mse = mean_squared_error(y_pred_train,y_train)
print("MSE: %2f" %mse)

MSE: 0.244482


In [38]:
y_pred_test = best_estm.predict(X_test)
mse = mean_squared_error(y_pred_test,y_test)
print("MSE: %2f" %mse)

MSE: 0.262032


Print classification Report

In [39]:
from sklearn.metrics import confusion_matrix,classification_report

In [40]:
cm = confusion_matrix(y_test,y_pred_test)

In [41]:
print(cm)

[[250 129]
 [ 67 302]]


In [42]:
print(classification_report(y_test,y_pred_test))

              precision    recall  f1-score   support

           0       0.79      0.66      0.72       379
           1       0.70      0.82      0.76       369

    accuracy                           0.74       748
   macro avg       0.74      0.74      0.74       748
weighted avg       0.75      0.74      0.74       748



Having adjusted my class size using random undersampler, the classification of those that churn has improved from 55% to 75%.  I also incorporated feature selection into the pipeline model to reduce unnecessary noise.